# Reservoirs


Reservoir operations include releasing water to satisfy demands within associated command areas and releasing water downstream. Demand-related releases are determined by matching demand requests up to a maximum. The requests to the reservoir are the total of irrigation and non-irrigation water demands within the command area of the associated reservoir. The maximum allowable releases are reservoir and day-speicific. There are 47 Managed and 215 unmanaged waterbodies are included in the Bhima example.

#### Player's guide for the Bhima basin:
✔️ update the file path below <br>
⏩ Run with the black and white fast-forward icon above.


In [1]:
output_folder = r'P:/watmodel/CWATM/modelruns/norway_1km/output_MS2'

In [2]:
run functions/functions_Maps.ipynb

In [3]:
display_map_noTime('waterBodyID_totalend',  -1, density=True, colourScale="agsunset", Rad=3, dark=False, daySum = True, colourRange=[0,1], outputFolder = output_folder) #'C:\CWatM_output' _Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'

FileNotFoundError: [Errno 2] No such file or directory: b'P:/watmodel/CWATM/modelruns/norway_1km/output_MS2/waterBodyID_totalend.nc'

In [4]:
from netCDF4 import Dataset, num2date
import plotly.graph_objects as go
import numpy as np
import datetime, os

In [5]:
Vars = [['lakeResStorage', 'Lake Level', 'Reservoir Volume', 'Volume (MCM)', 1000000.],
        ['lakeResOutflowM', 'Spilling', 'Reservoir Outflow', 'metres', 1.], 
        ['lakeResInflowM', 'positive', 'Reservoir Inflow', 'metres', 1.]]

        #['abstractedLakeReservoirM3', 'Abstractions', 'Abstractions', 'Volume (MCM)', 1000000.],
        #['reservoir_transfers_net_M3', 'Reservoir transfers (net)','Reservoir transfers (net)', 'Volume (MCM)', 1000000.],
        #['reservoir_transfers_to_outside_M3', 'Reservoir transfers to outside','Reservoir transfers to outside', 'Volume (MCM)', 1000000.],
        #['reservoir_transfers_from_outside_M3', 'Reservoir transfers from outside','Reservoir transfers from outside', 'Volume (MCM)', 1000000.]]

SIMULATED_nc = []
for var in Vars:
    reservoir_nc_filename = output_folder +'/'+ var[0] + '_daily.nc'
    SIMULATED_nc.append(Dataset(reservoir_nc_filename, 'r'))

In [6]:
Dates_simulation = num2date(SIMULATED_nc[0].variables['time'][:], units=SIMULATED_nc[0].variables['time'].units)
for t in range(len(Dates_simulation)):
    Dates_simulation[t] = np.datetime64(Dates_simulation[t])

In [11]:
Volumes1 = Dataset(output_folder +'/resVolume_totalend.nc', 'r')
Volume = Volumes1.variables['resVolume_totalend'][:,:]
reservoirs = []
try:
    for i in range(Volumes1.variables['lat'].shape[0]):
        for j in range (Volumes1.variables['lon'].shape[0]):
            if Volume[i,j] != 0:   
                reservoirs.append([Volume[i,j], i, j])
except:
    for i in range(Volumes1.variables['y'].shape[0]):
        for j in range (Volumes1.variables['x'].shape[0]):
            if Volume[i,j] != 0:   
                reservoirs.append([Volume[i,j], i, j])

In [12]:
Dams_all_names = [str(i[1])+', '+str(i[2]) for i in reservoirs]
Dams_names = Dams_all_names
Inds=[(i[1], i[2]) for i in reservoirs]

In [13]:
DAMS = []

for i in range(len(Vars)): 
    Dams = []
    for inds in Inds:
        Dams.append(SIMULATED_nc[i].variables[Vars[i][0]][:,inds[0], inds[1]]/Vars[i][4])
    DAMS.append(Dams)
    

In [14]:
for i in range(len(Vars)):
    
    fig = go.Figure()
    Dams = DAMS[i]
    
    for dam_i in range(len(Dams)):
        fig.add_trace(go.Scatter(y=Dams[dam_i],
                                 x=Dates_simulation,
                        mode='lines',
                        name=Dams_names[dam_i]))


    fig.update_layout(title = Vars[i][2] +', Simulated',
                           xaxis_title='Days',
                           yaxis_title= Vars[i][3],template='plotly_dark')

    fig.show()

## CWatM Outputs

The following CWatM outputs (in *italics*) are suggested. All of these variables must be output to display the above figures. 

Output | Description
--- | --- 
*lakeResInflowM* | Waterbody inflow (metres)
*lakeResInflowDis* | Waterbody inflow ($m^3/s$)
*EvapWaterBodyM* |  Waterbody evaporation (metres)
*act_bigLakeResAbst*| Waterbody abstractions conveyed to cell (metres)
*act_bigLakeResAbst_alloc* | Waterbody abstractions, from reservoirs 
*act_LocalLakeAbstractM3* | Basin transfer/local abstractions ($m^3$)
*leakage*| Canal leakage (from reservoirs) ($m^3$)
*lakeResOutflowM*| Waterbody outflow (metres)
*lakeResInflowDis*| Waterbody outflow ($m^3/s$)
*lakeResStorage*| Waterbody storage ($m^3$) 

## CWatM Inputs

-**Reservoir_releases_input_file** holds three variables, *Downstream_release*, *Fraction_of_Volume*, and *Basin Transfer* each with 366 timesteps. Each holds the maximum fraction of avaiable reservoir storage that is available daily for either downstream releases, command area requests, or transfers out of the basin (essentially water that is consumed at the reservoir). Please refer to the example input file as well as the Waterbodies Excel stating the NetCDF input coordinate associated with each reservoir. 

-**waterBodyElec** is a map with the value 1 in cells representing reservoirs transferring water out of the basin. 

-**Water_conveyance_efficiency** is either a single value or a map representing the water conveyenace efficiencey of different reservoirs. This map is to be similar to the command areas map, and the efficiency of the reservoir is held in each cell of the associated command area. 

-**floodStorageLimit_JuneAndEarlier** is a value [0,1] such that above this fraction of reservoir volume, the reservoir relases water downstream, for the months January to June. 

-**floodStorageLimit_July**  is a value [0,1] such that above this fraction of reservoir volume, the reservoir relases water downstream, in July. 

-**floodStorageLimit_AugustAndLater** is a value [0,1] such that above this fraction of reservoir volume, the reservoir relases water downstream, for the months August to December.


## Example

Reservoir A has an *available storage* of 200 MCM on day 1. The variables *Downstream_release* and *Fraction_of_Volume* on day 1 both equal 0.01. 


The associated command area is 6 cells: 4 cells for the footprint of the reservoir, and 2 other cells, Cell 1 and Cell 2. Cell 1 requests 1 MCM from the reservoir, Cell 2 requests 3 MCM from the reservoir, and the footprint cells do not request water from the reservoir. 

Then, command area request = 4 MCM >= *available storage* * *Fraction_of_Volume* = 0.01 * 200 = 2 MCM. Therefore , 2MCM is allocated to satisfy command area requests. 

Then, *available storage* * *Fraction_of_Volume* / *command area request* = 50% of all command area requests are satisfied. Therefore, Cell 1 is given 0.5 MCM and Cell 2 is given 1.5 MCM.

Reservoir A now has 198 MCM, and releases 198* *Downstream_release* = 198*0.01 = 1.98 MCM downstream.
